In [1]:
# Setup
bind 'set enable-bracketed-paste off'

In [2]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

In [3]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [4]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [5]:
# Setting a temp directory for tests
TMP=/tmp/test-cond-env
## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
## source $TEMP_BIN/_dir-aliases.bash
source _dir-aliases.bash
alias | wc -l

8


In [6]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1710)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1710

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-cond-env/test-1710


In [7]:
# Count aliases proper
alias | wc -l

9


In [8]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

2


In [9]:
# CONDITIONAL ENVIRONMENT SETTINGS - tomohara-aliases.sh
# Conditional environment variable setting
# Format: cond-export VAR1 VALUE1 [VAR2 VALUE2] ...
# EX: export FU="bar"; conditional-export FU baz; echo $FU => bar
#
function conditional-export () {
    local var value
    local args
    while [ "$1" != "" ]; do
        var="$1" value="$2";
        ## DEBUG: echo "value for env. var. $var: $(printenv "$var")"
        if [ "$(printenv "$var")" == "" ]; then
	    # Note: ignores SC1066 (Don't use $ on the left side of assignments)
	    # shellcheck disable=SC1066      
            export $var="$value"
        fi
	args="$*"
        shift 2
	if [ "$args" = "$*" ]; then
	    echo "Error: Unexpected value in conditional-export (var='$var'; val='$value')"
	    return 
	fi
    done
}

In [10]:
# SETTING ALIASES FOR CONDITIONAL EXPORT
alias conditional-setenv='conditional-export'
alias cond-export='conditional-export'
alias cond-setenv='conditional-export'

In [11]:
# SETTING A ENVIRONMENT VARIABLE NAMED COND_ENV_TEST_TEMP
cond-setenv COND_ENV_TEST_TEMP tmp/cond-env-test-temp

In [12]:
# PROOF THAT THE ENVIRONMENT VARIABLE IS CREATED
echo $COND_ENV_TEST_TEMP

tmp/cond-env-test-temp


In [13]:
# USE OF ENV VARIABLE TO CREATE A SUB-DIRECTORY
mkdir -p $COND_ENV_TEST_TEMP
ls -l | awk '!($6=$7=$8="")'

total 4      
drwxrwxr-x 3 aveey aveey 4096    tmp


In [14]:
# THE OUTPUT OF printenv.sh IS STORED IN temp_env1.tmp CREATED IN THE DIRECTORY SPECIFIED BY $COND_ENV_TEST_TEMP
pwd
bash $BIN_DIR/printenv.sh > $COND_ENV_TEST_TEMP/temp-env-1

/tmp/test-cond-env/test-1710


In [15]:
# MOVING TO THE DIRECTORY & VIEWING THE OUTPUT LOG
cd $COND_ENV_TEST_TEMP

# ERROR PRONE COMMAND (BATSPP)
# cat temp-env-1